<a href="https://colab.research.google.com/github/4en1x/diploma/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install gcc-4.8 g++-4.8 -y -qq;
!ln -s /usr/bin/gcc-4.8 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-4.8 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

38 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-03-02 22:13:27--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?HX0MMkZWYCvehYi-cABK6Il4nGrRhqYmJ4pnYbE5Imihxup9s7y6NEmbVuHkWW2CZGluSVRgf0sknnaF2VCtRBjw1pNXDGnAJ2T2r_6lla7-AtY98gjoslJNIEoG3Zno4dj2iXwpn3AtFG9gHTAUTR9FXrC74bzPKq9OUeulICKnpVeYCGh9Lz7e9nbrTnIFmu3yNNSEQsfVNWryoKEnfwSJWw [following]
--2019-03-02 22:13:28--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?HX0MMkZWYCvehYi-cABK6I

In [0]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-o5ri9uvo
  Stored in directory: /tmp/pip-ephem-wheel-cache-kz569vqn/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
!cd /usr/src
!ln -sfn linux-headers-2.6.35-28-generic linux

CODE (AT LEAST NOW)

In [62]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 100
#define N 32

#define e 2.7182818

#define max(x, y) (((x) > (y)) ? (x) : (y))
#define min(x, y) (((x) < (y)) ? (x) : (y))


#define blockZSize 2

#define blockXSize 16
#define blockYSize 16

#define tileXSize 8
#define tileYSize 8

#define extN (N + 2 * (blockZSize - 1))
#define Dx 1
#define h ((float) Dx / extN)
#define h2 powf(h, 2.0)

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
float u(float* matrix, int i, int j) {
	return matrix[extN * i + j];
}

__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + 1;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + 1;
  // And now for something completely different
  int N1 = tileXSize + blockZSize * 2;
  int N2 = tileYSize + blockZSize * 2;
  float* helper = new float[N1 * N2];
  
  for(int i = 0; i < tileXSize + blockZSize; i++) {
		for(int j = 0; j < tileYSize + blockZSize; j++) {
			helper[i * N1 + j] = matrix[extN * (iBegin - blockZSize + i) + jBegin - blockZSize + j];
		}
	}
  
    
  for(int k = 1; k < blockZSize; k++) {
		for (int i = k; i < N1 - k; i++) {
		  int offset = i % 2;
		  if (isBlack) {
			  offset = 1 - offset;
		  }
		  for (int j = k; j < N2 - k; j += 2) {
			  helper[N1 * i + j] = 0.25 * (
            helper[N1 * (i + 1) + j]
          + helper[N1 * (i - 1) + j]
          + helper[N1 * i + j + 1]
				  + helper[N1 * i + j - 1] 
          - h2 * f(iBegin - blockZSize + i, jBegin - blockZSize + j)
        );
		  }
	  }
	}
  
    
  //
    

  //printf("iGl : %d, %d, %d, %d, %d, %d, %d \n", iGl, jGl, threadIdx.x, threadIdx.y, isBlack, iBegin, jBegin);

	for (int i = iBegin; i < iBegin + tileXSize; i++) {
		int offset = i % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
		for (int j = jBegin + offset; j < jBegin + tileYSize; j += 2) {
			matrix[extN * i + j] = 0.25 * (
          helper[N1 * (i + 1 - iBegin) + j - jBegin]
        + helper[N1 * (i - 1 - iBegin) + j - jBegin]
        + helper[N1 * (i - iBegin) + j + 1 - jBegin]
				+ helper[N1 * (i - iBegin) + j - 1 - jBegin] 
        - h2 * f(i, j)
      );
		}
	}
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];

	for(int i = 0; i < row; i++) {
		for(int j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}

  for(int k = 0; k < blockZSize; k++) {
    for(int i = k; i < row - k; i++) {
      matrix[k * row + i] = u_t((i - blockZSize + 1) * h);
      matrix[(row - 1 - k) * row + i] = u_d((i - blockZSize + 1) * h);
    }
	}

  for(int k = 0; k < blockZSize; k++) {
    for(int j = k; j < col - k; j++) {
      matrix[j * col + k] = u_l((j - blockZSize + 1) * h);
      matrix[j * col + row - 1 - k] = u_r((j - blockZSize + 1) * h);
	  }
  }
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;

  for (int i = blockZSize; i < extN - blockZSize; i++) {
		for (int j = blockZSize; j < extN - blockZSize; j++) {
      float fCurrent = func(i, j);
		  tolerance += (matrix1[i * extN + j] - fCurrent) * (matrix1[i * extN + j] - fCurrent);
		}
	}

	return tolerance;
}

int main() {
  ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(extN, extN);
  

  for (size_t i = 0; i < extN; i++) {
		for (size_t j = 0; j < extN; j++) {
      myfile << matrixHost[i * extN + j] << " ";
		}
    myfile << "\n";
	}



	float* matrixDevice;
	
	int matrixSize = sizeof(float) * extN * extN;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / blockXSize, N / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);

		cudaPeekAtLastError();
		cudaDeviceSynchronize();

		cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

      
    cout << calculateDifference(matrixHost) << "\n";
   
    for (size_t i = 0; i < extN; i++) {
		  for (size_t j = 0; j < extN; j++) {
        myfile << matrixHost[i * extN + j] << " ";
		  }
      myfile << "\n";
	 }
   
     
	}
	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}
  myfile.close();
	return 0;
}


'7547.51\n7088.41\n6649.01\n6238.12\n5871.11\n5565.49\n5325.87\n5153.19\n5046.24\n4993.36\n4973.48\n4967.94\n4966.8\n4966.64\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\n4966.62\nCalculation take time: 503903\n'

In [64]:
import os

f = open("example.txt", 'r')
i = 0

content = f.readlines()
for line in content:
  if i % 34 == 0:
    print('\n')
  line = line.split(' ')
  for el in line:
    print(el.rjust(12), end='')
  i += 1




    0.971017           1     1.02985     1.06059     1.09225     1.12485     1.15842       1.193     1.22861     1.26528     1.30305     1.34194       1.382     1.42325     1.46573     1.50948     1.55454     1.60094     1.64872     1.69793     1.74861     1.80081     1.85456     1.90992     1.96692     2.02563      2.0861     2.14836     2.21249     2.27853     2.34654     2.41658     2.48871      2.6395           
           1           1     1.02985     1.06059     1.09225     1.12485     1.15842       1.193     1.22861     1.26528     1.30305     1.34194       1.382     1.42325     1.46573     1.50948     1.55454     1.60094     1.64872     1.69793     1.74861     1.80081     1.85456     1.90992     1.96692     2.02563      2.0861     2.14836     2.21249     2.27853     2.34654     2.41658     2.71828     2.71828           
     1.02985     1.02985           0           0           0           0           0           0           0           0           0           0           0  

KeyboardInterrupt: ignored